<center><font size=5 ><strong>Homework 4</strong></font></center>

### **1. 题目说明**

下表左侧有18个词性序列模式（POS Sequence patterns），右侧是每个模式在文本中对应的实例，`Freq.`列内容可忽略。例如，模式1`Noun Noun`对应的一个实例为`Group conversation`。

<div align=center>
<img width="850" height="350" src="https://github.com/zhangjianzhang/text_mining/blob/master/files/codes/lecture_7/patterns.jpg?raw=true">
<br>
<center><em><strong>POS sequence patterns and instances</strong></em></center>
</div>

### **2. 题目要求**

对给定的一个文本集合`text.csv`，下载地址如下：

https://raw.githubusercontent.com/zhangjianzhang/text_mining/master/files/homework/text.csv

从中抽取出符合上图中18个pattern的全部实例。

例如，给定输入文本 ***you can send beautiful pictures anywhere.*** ，其中`beautiful pictures`符合pattern 3`Adjective Noun`，`send beautiful pictures`符合pattern 9`Verb Adjective Noun`

<font style="color:#FF0000">**注意**</font>：将全部匹配结果输出为一个字典，可以将该字典写入本地`json`文件，格式如下：
```python
{
    '3':['beautiful picture']
    '9':['send beautiful picture']
}
```
key表示上图中pattern的编号，value为实例列表，列表中的实例需要是**小写**且**词形还原**的形式（lowercase lemma）

### **3. 提示**

- 分句，分词，词性标注，小写化，词形还原（lemmatization），词性序列匹配，抽取实例并保存；

- 上述文本处理操作均可借助NLTK完成，当然，你也可以借助其他文本处理包（如 spacy, stanza等）实现上述文本处理步骤；

- 可以参考课件、课程配套代码、互联网搜索资料。

- nltk默认词性标注器使用的词性标签的含义：

   - https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
   - 或者执行代码`nltk.help.upenn_tagset()`来查询

In [1]:
# 请直接在该jupyter文件中写你的答案，并在截止日之前提高该jupyter

# Please write your codes below

In [2]:
# 导入所需的包
# fileUtils包是我自己写的文件读写模块，可以从课程网站下载
# 将解压后得到的utils文件夹，放到目录 anaconda3/lib/python3.9/ 中即可
import nltk
from utils.fileUtils import *
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# lemmatization
wnl = WordNetLemmatizer()

# 将词性标签映射为WordNet lemmatizer接受的词性标签
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [3]:
# 生成N-gram
def get_n_gram(text_list,n):
    grams = []
    m = len(text_list) - n + 1
    for idx in range(m):
        gram = text_list[idx:idx+n]
        grams.append(gram)
    return grams
# sample_text = 'so much for making this app such an enjoyable'.split()
# get_n_gram(sample_text,1)
# get_n_gram(sample_text,2)
# get_n_gram(sample_text,3)

In [4]:
# 为便于匹配，将pattern中的词性标签进行映射
def pos_tag_map(pos_tag):
    # 名词
    if pos_tag.startswith('N'):
        return 'N'
    # 动词
    elif pos_tag.startswith('V'):
        return 'V'
    # 形容词
    elif pos_tag.startswith('J'):
        return 'J'
    # 代词
    elif pos_tag in ['PRP','PRP$','WP','WP$']:
        return 'P'
    # 连词 CC，介词IN
    else:
        return pos_tag

In [5]:
# 分词，词性标注，词形还原和小写化 (word tokenization, POS tagging, lemmatization and lowercase)
def text_processing(text):
    sentences = nltk.sent_tokenize(text)
    sent_lists = []
    for idx,sent in enumerate(sentences):
        sent_list = [(wnl.lemmatize(w,get_wordnet_pos(t)).lower(),pos_tag_map(t)) for w,t in nltk.pos_tag(nltk.word_tokenize(sent))]
        sent_lists.append(sent_list)
    return sent_lists

In [6]:
# 定义pattern
patterns = [['N','N'],
           ['V','N'],
           ['J','N'],
           ['N','CC','N'],
           ['J','N','N'],
           ['N','N','N'],
           ['V','P','N'],
           ['V','N','N'],
           ['V','J','N'],
           ['J','J','N'],
           ['N','IN','N'],
           ['V','DT','N'],
           ['V','N','IN','N'],
           ['J','N','N','N'],
           ['J','CC','J'],
           ['V','IN','J','N'],
            ['V','P','J','N'],
           ['N','CC','N','N']]

patterns_dict = {k+1:v for k,v in enumerate(patterns)}

In [7]:
# 定义结果字典
result_dict = {k+1:set() for k in range(len(patterns))}

In [8]:
# 读取待处理数据
lines = readCsvToList('./text.csv')

In [9]:
# 模式匹配，将匹配到的实例存入result_dict
for line in lines:
    tokenized_sents = text_processing(line[0])
    for tokenized_sent in tokenized_sents:
        gram_2 = get_n_gram(tokenized_sent, 2)
        gram_3 = get_n_gram(tokenized_sent, 3)
        gram_4 = get_n_gram(tokenized_sent, 4)
        for item in gram_2 + gram_3 + gram_4:
            pos_list = []
            word_list = []
            for word_tuple in item:
                word, pos = word_tuple
                pos_list.append(pos)
                word_list.append(word)
            for k,pattern in patterns_dict.items():
                if pos_list == pattern:
                    result_dict[k].add(' '.join(word_list))

In [10]:
# 将result_dict保存到本地json文件中
dumpJson('./pattern_match_result.json', {k:list(v) for k,v in result_dict.items()})

In [ ]:
# 对上述匹配结果还可以进行进一步改进，比如：
# 1. 使用更准确的词性标注器，spacy
# 2. 取出匹配结果中的噪音
# 3. 对于嵌套的模式进行去重